In [1]:
import sys, os
sys.path.insert(0, '..')

In [2]:
import graph_tool.all as gt
import graph_embeddings
from models.crosswalk import Crosswalk
from utils.score import statistical_parity
import faiss
import residual2vec as rv
import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

from sklearn.neighbors import kneighbors_graph
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

from utils import node_classification
from torch.utils.data import DataLoader


Bad key "text.kerning_factor" on line 4 in
/home/ashutosh/miniconda3/envs/study/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
2022-08-17 18:26:09.723182: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-08-17 18:26:09.748714: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2304000000 Hz
2022-08-17 18:26:09.749897: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55eafb7e5250 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-17 18:26:09.749915: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-17 18:26:09

In [3]:
models = {}
window_length = 5
num_walks = 10
dim = 128
walk_length = 80

In [4]:
DATA_FILE = '../data/polbooks.gml'
G = nx.read_gml(DATA_FILE)
G = nx.relabel.convert_node_labels_to_integers(G, first_label=0, ordering='default')

nodes = G.nodes(data=True)
labels, group_ids = np.unique([n[1]['value'] for n in nodes], return_inverse=True)

A = nx.adjacency_matrix(G).asfptype()
deg = np.array(A.sum(axis=1)).reshape(-1)
G = nx.from_scipy_sparse_matrix(A)

In [5]:
models["degree-unbiased"] = rv.residual2vec_sgd(
    noise_sampler=rv.ConfigModelNodeSampler(),
    window_length=window_length,
    num_walks=num_walks,
    walk_length=walk_length
).fit(A)

models["group-unbiased"] = rv.residual2vec_sgd(
    noise_sampler=rv.SBMNodeSampler(
        group_membership=group_ids, window_length=window_length,
    ),
    window_length=window_length,
    num_walks=num_walks,
    walk_length=walk_length,
).fit(A)


In [6]:
embs = {}
for k, model in models.items():
# needs to take weighted adjacency amtrix and then pass torch model and dataset
    from residual2vec.word2vec import Word2Vec
    adjusted_num_walks = np.ceil(
            num_walks
            * np.maximum(
                1,
                model.batch_size
                * model.miniters
                / (model.n_nodes * num_walks * walk_length),
            )
        ).astype(int)
    dataset = rv.TripletDataset(
            adjmat=model.adjmat,
            num_walks=adjusted_num_walks,
            window_length=model.window_length,
            noise_sampler=model.sampler,
            padding_id=model.n_nodes,
            walk_length=model.walk_length,
            p=model.p,
            q=model.q,
            buffer_size=model.buffer_size,
            context_window_type=model.context_window_type,
        )
    dataloader = DataLoader(
            dataset,
            batch_size=model.batch_size,
            shuffle=False,
            num_workers=1,
            pin_memory=True,
        )

    m = Word2Vec(vocab_size=A.shape[0] + 1, embedding_size=dim, padding_idx=A.shape[0])
    embs[k] = model.transform(model=m, dataloader=dataloader)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 329/329 [00:00<00:00, 463.49it/s, loss=1.21]
